In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [5]:
# Load the datasets
train_df = pd.read_csv('/content/drive/MyDrive/FYP_2024/Final_Train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/FYP_2024/Final_test.csv')
validation_df = pd.read_csv('/content/drive/MyDrive/FYP_2024/Final_Validation.csv')

In [6]:
# Function to preprocess datasets
def preprocess_data(df):
    df['local_time'] = pd.to_datetime(df['local_time'])
    for time_unit in ['Year', 'Month', 'Day', 'Hour']:
        df[time_unit] = getattr(df['local_time'].dt, time_unit.lower())
    return df.drop('local_time', axis=1)

In [7]:
def add_cyclical_features(df, col_name, max_val):
    if col_name in df.columns:
        df[col_name + '_sin'] = np.sin(2 * np.pi * df[col_name] / max_val)
        df[col_name + '_cos'] = np.cos(2 * np.pi * df[col_name] / max_val)
    return df

#'Hour', 'Day', and 'Month' created
for df in [train_df, validation_df, test_df]:
    df = add_cyclical_features(df, 'Hour', 23)
    df = add_cyclical_features(df, 'Day', 31)
    df = add_cyclical_features(df, 'Month', 12)

In [8]:
def add_lagged_features(df, feature_cols, n_lags=3):
    for feature in feature_cols:
        for lag in range(1, n_lags + 1):
            df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
    return df.dropna().reset_index(drop=True)

# Preprocess all datasets
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
validation_df = preprocess_data(validation_df)

# Specifying the features to lag
features_to_lag = ['Average_Temp', 'MW']  # Add other relevant features as needed

# Applying the function to the datasets
train_df = add_lagged_features(train_df, features_to_lag, n_lags=3)
validation_df = add_lagged_features(validation_df, features_to_lag, n_lags=3)
test_df = add_lagged_features(test_df, features_to_lag, n_lags=3)

In [9]:
from sklearn.preprocessing import PolynomialFeatures

# Creating polynomial and interaction features
poly = PolynomialFeatures(degree=2, include_bias=False)

# Applying transformation to all datasets.
train_features = train_df.drop(columns=['MW'])
validation_features = validation_df.drop(columns=['MW'])
test_features = test_df.drop(columns=['MW'])

X_train_poly = poly.fit_transform(train_features)
X_validation_poly = poly.transform(validation_features)
X_test_poly = poly.transform(test_features)


feature_names = poly.get_feature_names_out(input_features=train_features.columns)


y_train = train_df['MW']
y_validation = validation_df['MW']
y_test = test_df['MW']

In [10]:
# Evaluation Function

def calculate_metrics(actual, predicted, lower_bound=0, upper_bound=100, iqr_multiplier=1.5):
    # Excluding negative actual values if considered invalid
    valid_indices = actual > lower_bound
    actual = actual[valid_indices]
    predicted = predicted[valid_indices]

    # Calculate MAE and RMSE
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))

    # Thresholding for outlier exclusion based on IQR
    q1, q3 = np.percentile(actual, [25, 75])
    iqr = q3 - q1
    outlier_threshold_upper = q3 + (iqr * iqr_multiplier)
    outlier_threshold_lower = q1 - (iqr * iqr_multiplier)

    valid_indices_for_mape = (actual >= outlier_threshold_lower) & (actual <= outlier_threshold_upper)
    filtered_actual = actual[valid_indices_for_mape]
    filtered_predicted = predicted[valid_indices_for_mape]

    # Calculate Modified MAPE with capped at 100%
    if len(filtered_actual) > 0:
        percentage_errors = np.abs((filtered_predicted - filtered_actual) / filtered_actual) * 100
        percentage_errors = np.clip(percentage_errors, None, upper_bound)  # Cap percentage errors at upper_bound (100%)
        mape = np.mean(percentage_errors)
    else:
        mape = np.nan

    # Calculate sMAPE
    smape = 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

    return mae, mape, smape, rmse

In [11]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
import numpy as np

def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch / step_size))
    return LearningRateScheduler(schedule)

# Ensure X_train_poly, X_validation_poly, and X_test_poly are numpy arrays for LSTM
X_train_lstm = np.array(X_train_poly)
X_validation_lstm = np.array(X_validation_poly)
X_test_lstm = np.array(X_test_poly)

# Reshape the data for LSTM [samples, time steps, features]
# Assuming that we are treating each sample as an independent observation with 1 time step
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], 1, X_train_lstm.shape[1]))
X_validation_lstm = X_validation_lstm.reshape((X_validation_lstm.shape[0], 1, X_validation_lstm.shape[1]))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], 1, X_test_lstm.shape[1]))

# Confirming the shape
print("X_train shape:", X_train_lstm.shape)
print("X_validation shape:", X_validation_lstm.shape)
print("X_test shape:", X_test_lstm.shape)

# Model definition
model = Sequential([
    LSTM(100, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    BatchNormalization(),
    Dense(1, kernel_regularizer=l2(0.01))
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Callbacks
checkpoint_path = '/content/drive/MyDrive/Saved_trained_models/best_rnn_model_V3.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
lr_scheduler = step_decay_schedule(initial_lr=0.001, decay_factor=0.75, step_size=10)

# Model training
history = model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_validation_lstm, y_validation),
    callbacks=[checkpoint, early_stopping, lr_scheduler],
    verbose=1
)

# Load the best model after training
model.load_weights(checkpoint_path)
print("Model training complete. Best model loaded.")

model.save(checkpoint_path)
print("Model saved at: {}".format(checkpoint_path))


X_train shape: (17516, 1, 77)
X_validation shape: (8758, 1, 77)
X_test shape: (1437, 1, 77)
Epoch 1/50
271/274 [============================>.] - ETA: 0s - loss: 8460.3750
Epoch 1: val_loss improved from inf to 3989.94751, saving model to /content/drive/MyDrive/Saved_trained_models/best_rnn_model_V3.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


274/274 [==============================] - 8s 18ms/step - loss: 8433.1094 - val_loss: 3989.9475 - lr: 0.0010
Epoch 2/50
269/274 [============================>.] - ETA: 0s - loss: 5843.9248
Epoch 2: val_loss improved from 3989.94751 to 2107.23267, saving model to /content/drive/MyDrive/Saved_trained_models/best_rnn_model_V3.h5
274/274 [==============================] - 2s 9ms/step - loss: 5855.5244 - val_loss: 2107.2327 - lr: 0.0010
Epoch 3/50
269/274 [============================>.] - ETA: 0s - loss: 4229.1870
Epoch 3: val_loss improved from 2107.23267 to 1915.45935, saving model to /content/drive/MyDrive/Saved_trained_models/best_rnn_model_V3.h5
274/274 [==============================] - 2s 8ms/step - loss: 4213.5562 - val_loss: 1915.4594 - lr: 0.0010
Epoch 4/50
272/274 [============================>.] - ETA: 0s - loss: 3985.0708
Epoch 4: val_loss did not improve from 1915.45935
274/274 [==============================] - 2s 6ms/step - loss: 3974.2019 - val_loss: 1977.6581 - lr: 0.0010

In [12]:
from keras.models import load_model

# Load the trained model
model = load_model('/content/drive/MyDrive/Saved_trained_models/best_rnn_model_V3.h5')

In [14]:
import numpy as np

# Ensure X_validation_poly is a numpy array
X_validation_poly_array = np.array(X_validation_poly)

# Reshape X_validation_poly for LSTM
# Adding the time step dimension
X_validation_poly_reshaped = X_validation_poly_array.reshape((X_validation_poly_array.shape[0], 1, X_validation_poly_array.shape[1]))

# Now you can predict using the reshaped validation data
predictions = model.predict(X_validation_poly_reshaped)


274/274 [==============================] - 2s 5ms/step


In [16]:
import numpy as np

# Ensure X_train_poly is a numpy array
X_train_poly_array = np.array(X_train_poly)

# Reshape X_train_poly for LSTM
# Adding the time step dimension
X_train_poly_reshaped = X_train_poly_array.reshape((X_train_poly_array.shape[0], 1, X_train_poly_array.shape[1]))

# Predict using the reshaped training data
train_predictions = model.predict(X_train_poly_reshaped)

# Flatten predictions to match the shape of y_train for evaluation
train_predictions_flattened = train_predictions.flatten()

# Evaluate the model on the training set using your custom function
train_mae, train_mape, train_smape, train_rmse = calculate_metrics(y_train, train_predictions_flattened)

# Print the evaluation metrics for the training set
print(f"Training Set - MAE: {train_mae}, MAPE: {train_mape}, sMAPE: {train_smape}, RMSE: {train_rmse}")


548/548 [==============================] - 3s 5ms/step
Training Set - MAE: 31.733617338091808, MAPE: 42.23611141566295, sMAPE: 42.14225898776994, RMSE: 61.038844805006136


In [17]:
import numpy as np

# Assuming you've already made predictions and have your model and data ready

# Ensure predictions are 1D
predictions = predictions.squeeze()

# If necessary, also ensure y_validation is 1D
# This line may be unnecessary if y_validation is already 1D
y_validation_array = np.array(y_validation)  # Ensure y_validation is a numpy array
y_validation_squeezed = y_validation_array.squeeze()

# Now, evaluate the model using your custom function
mae, mape, smape, rmse = calculate_metrics(y_validation_squeezed, predictions)

# Print the evaluation metrics
print(f"MAE: {mae}, MAPE: {mape}, sMAPE: {smape}, RMSE: {rmse}")


MAE: 29.814441659553065, MAPE: 47.41280686745494, sMAPE: 47.71087478952085, RMSE: 42.93988185280797


In [18]:
import numpy as np

# Assuming your model is already trained and you have the test features and labels ready

# Ensure X_test_poly is a numpy array
X_test_poly_array = np.array(X_test_poly)

# Reshape X_test_poly for LSTM [samples, time steps, features]
X_test_poly_reshaped = X_test_poly_array.reshape((X_test_poly_array.shape[0], 1, X_test_poly_array.shape[1]))

# Predict using the reshaped test data
test_predictions = model.predict(X_test_poly_reshaped)

# Ensure predictions are 1D
test_predictions_squeezed = test_predictions.squeeze()

# If necessary, also ensure y_test is 1D
# This line may be unnecessary if y_test is already 1D
y_test_array = np.array(y_test)  # Ensure y_test is a numpy array
y_test_squeezed = y_test_array.squeeze()

# Now, evaluate the model using your custom function
test_mae, test_mape, test_smape, test_rmse = calculate_metrics(y_test_squeezed, test_predictions_squeezed)

# Print the evaluation metrics for the test set
print(f"Test Set - MAE: {test_mae}, MAPE: {test_mape}, sMAPE: {test_smape}, RMSE: {test_rmse}")


45/45 [==============================] - 1s 16ms/step
Test Set - MAE: 28.294001512065865, MAPE: 47.200871079531794, sMAPE: 42.48477458613127, RMSE: 38.941466614452814
